# Synthetic Log Generation from DECLARE Models

DECLARE4Py implements the generation of synthetic logs from DECLARE models with a solution based on Answer Set Programming that uses a Clingo solver. More details can be found in the paper of Chiariello, F., Maggi, F. M., & Patrizi, F. (2022, June). ASP-Based Declarative Process Mining. In *Proceedings of the AAAI Conference on Artificial Intelligence* (Vol. 36, No. 5, pp. 5539-5547).

As first step, it is necessary to import a `.decl` file containing the DECLARE constraints.

In [2]:
import os

from Declare4Py.ProcessModels.DeclareModel import DeclareModel

model_name = 'sepsis'
model: DeclareModel = DeclareModel().parse_from_file(os.path.join("../../../", "tests", "test_models", f"{model_name}.decl"))

/usr/lib/python3/dist-packages/pyparsing.py:108: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


Then, some general settings are needed to set the number of cases to generate and the minimum and maximum number of events for each case

In [3]:
# Number of cases that have be generated
num_of_cases = 10

# Minimum and maximum number of events a case can contain
(num_min_events, num_max_events) = (3, 10)

The class `AspGenerator` has to be instantiated with the DECLARE model and the settings of above. Then, the `run` method will generate the cases and the `to_xes` method will save them in a `.xes` event log.

In [4]:
from Declare4Py.ProcessMiningTasks.ASPLogGeneration.asp_generator import AspGenerator

asp_gen: AspGenerator = AspGenerator(model, num_of_cases, num_min_events, num_max_events)
asp_gen.run()

DEBUG:ASP generator:Distribution for traces uniform
DEBUG:ASP generator:traces: 10, events can have a trace min(3) max(10)
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8)]
DEBUG:Distributor:Custom_dist() min_mu:3 max_sigma:10 num_traces:10
DEBUG:Distributor:Probabilities sum 1
DEBUG:Distributor:Distribution result: [ 8  7  5 10  9  5  3  9  7 10]
INFO:ASP generator:Distribution result Counter({7: 2, 5: 2, 10: 2, 9: 2, 8: 1, 3: 1})
DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fractio

In [5]:
asp_gen.to_xes(f'{model_name}.xes')

exporting log, completed traces ::   0%|          | 0/10 [00:00<?, ?it/s]

Logs can be generated with some **purposes** according to the needs of Process Mining algorithms. DECLARE4Py implements four useful purposes that can be set with the following methods of the `AspGenerator` class.


## 1. Setting up the Length Distribution of the Cases

Users can specify a probability distribution over the lengths of the generated traces. The method `set_distribution` takes as parameter the `distributor_type`. By setting this parameter with the `uniform` value, a uniform distribution in `[num_min_events, num_max_events]` is chosen.

In [6]:
asp_gen.set_distribution(distributor_type="uniform")

asp_gen.run()
asp_gen.to_xes(f'{model_name}.xes')

DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8)]
DEBUG:Distributor:Custom_dist() min_mu:3 max_sigma:10 num_traces:10
DEBUG:Distributor:Probabilities sum 1
DEBUG:Distributor:Distribution result: [3 7 8 7 8 7 9 5 9 6]
INFO:ASP generator:Distribution result Counter({7: 3, 8: 2, 9: 2, 3: 1, 5: 1, 6: 1})
INFO:ASP generator:Computing distribution
DEBUG:ASP generator:Generating traces
DEBUG:ASP generator:Translate declare model to ASP
DEBUG:ASP generator:Declare model translated to ASP. Total Facts 13
DEBUG:ASP generator:ASP encoding generated
DEBUG:ASP generator:Start generating traces: Counter({7: 3, 8: 2, 9: 2, 3: 1, 5: 1, 6: 1})
DEBUG:ASP generator: Total trace to generate and events:

exporting log, completed traces ::   0%|          | 0/10 [00:00<?, ?it/s]

A `gaussian` distribution requires a location (the mean) and a scale (the variance)

In [7]:
asp_gen.set_distribution(distributor_type="gaussian", loc=5, scale= 2)
asp_gen.run()
asp_gen.to_xes(f'{model_name}.xes')

DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
INFO:ASP generator:Computing gaussian distribution with mu=5 and sigma=2
DEBUG:Distributor:Distribution() gaussian min_mu: 5 max_sigma: 2 num_traces: 10 custom_prob: None
INFO:ASP generator:Gaussian distribution result Counter({5.0: 5, 3.0: 2, 6.0: 2})
INFO:ASP generator:Gaussian distribution after refinement {5.0: 5, 3.0: 2, 6.0: 2}
INFO:ASP generator:Distribution result {5.0: 5, 3.0: 2, 6.0: 2}
INFO:ASP generator:Computing distribution
DEBUG:ASP generator:Generating traces
DEBUG:ASP generator:Translate declare model to ASP
DEBUG:ASP generator:Declare model translated to ASP. Total Facts 13
DEBUG:ASP generator:ASP encoding generated
DEBUG:ASP generator:Start generating traces: {5.0: 5, 3.0: 2, 6.0: 2}
DEBUG:ASP generator: Total trace to generate and events: Traces:5, Events: 5.0, RandFrequency: 0.9
DEBUG:ASP generator: Generating trace:1/5 with events:5.0)
DEBUG:ASP generator: Initializing clingo 

exporting log, completed traces ::   0%|          | 0/9 [00:00<?, ?it/s]

A `custom` distribution requires the user to set the probability for each length in `[num_min_events, num_max_events]`

In [8]:
asp_gen.set_distribution(distributor_type="custom",
                         custom_probabilities=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.3])

asp_gen.run()
asp_gen.to_xes(f'{model_name}.xes')

DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() custom min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.3]
DEBUG:Distributor:Custom_dist() min_mu:3 max_sigma:10 num_traces:10
DEBUG:Distributor:Probabilities sum 1.0
DEBUG:Distributor:Distribution result: [ 7  8 10  4 10  6  5  8 10  3]
INFO:ASP generator:Distribution result Counter({10: 3, 8: 2, 7: 1, 4: 1, 6: 1, 5: 1, 3: 1})
INFO:ASP generator:Computing distribution
DEBUG:ASP generator:Generating traces
DEBUG:ASP generator:Translate declare model to ASP
DEBUG:ASP generator:Declare model translated to ASP. Total Facts 13
DEBUG:ASP generator:ASP encoding generated
DEBUG:ASP generator:Start generating traces: Counter({10: 3, 8: 2, 7: 1, 4: 1, 6: 1, 5: 1, 3: 1})
DEBUG:ASP generator: Total trace to generate and events: Traces:1, Events: 7, RandFrequency: 0.9
DEBUG:ASP generator: Generating trace:1/1 with events:7)
DEBUG:ASP gen

exporting log, completed traces ::   0%|          | 0/10 [00:00<?, ?it/s]

## 2. Setting up the Variants

Users can generate variants by setting the number of repetitions of the workflow of each case. This is done with the `set_number_of_repetition_per_trace` method

In [9]:
asp_gen.set_number_of_repetition_per_trace(3)

asp_gen.run()
asp_gen.to_xes(f'{model_name}.xes')

DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() custom min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.3]
DEBUG:Distributor:Custom_dist() min_mu:3 max_sigma:10 num_traces:10
DEBUG:Distributor:Probabilities sum 1.0
DEBUG:Distributor:Distribution result: [ 5  6  4  5  4  9  3  7 10  9]
INFO:ASP generator:Distribution result Counter({5: 2, 4: 2, 9: 2, 6: 1, 3: 1, 7: 1, 10: 1})
INFO:ASP generator:Computing distribution
DEBUG:ASP generator:Generating traces
DEBUG:ASP generator:Translate declare model to ASP
DEBUG:ASP generator:Declare model translated to ASP. Total Facts 13
DEBUG:ASP generator:ASP encoding generated
DEBUG:ASP generator:Start generating traces: Counter({5: 2, 4: 2, 9: 2, 6: 1, 3: 1, 7: 1, 10: 1})
DEBUG:ASP generator: Total trace to generate and events: Traces:2, Events: 5, RandFrequency: 0.9
DEBUG:ASP generator: Generating trace:1/2 with events:5)
DEBUG:ASP gen

exporting log, completed traces ::   0%|          | 0/30 [00:00<?, ?it/s]

## Setting up Positive and Negative Traces

Users can specify some constraints to be violated in the synthetic cases to obtain labelled logs for binary classification, e.g., for deviance mining algorithms. The method `set_constraints_to_violate` takes as input:

1. `tot_negative_trace`: the number of negative cases to be violated;
2. `violate_all`: whether to violate *all* the specified constraints or let Clingo decide which constraints to be violated;
3. `constraints_list`: the list containing the subset of DECLARE constraints (specified as strings of text) to be violated.

In [10]:
asp_gen: AspGenerator = AspGenerator(model, num_of_cases, num_min_events, num_max_events)

asp_gen.set_constraints_to_violate(tot_negative_trace=10, violate_all=True, constraints_list=[
    "Init[ER Registration] | |",
    "Chain Response[ER Registration, ER Triage] |A.org:group is J |T.org:group is A |"])
asp_gen.run()
asp_gen.to_xes(f'{model_name}.xes')

DEBUG:ASP generator:Distribution for traces uniform
DEBUG:ASP generator:traces: 10, events can have a trace min(3) max(10)
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8)]
DEBUG:Distributor:Custom_dist() min_mu:3 max_sigma:10 num_traces:10
DEBUG:Distributor:Probabilities sum 1
DEBUG:Distributor:Distribution result: [ 9  9 10  8  7 10  7 10  9  9]
INFO:ASP generator:Distribution result Counter({9: 4, 10: 3, 7: 2, 8: 1})
DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(

exporting log, completed traces ::   0%|          | 0/10 [00:00<?, ?it/s]

In addition, instead of giving the explicit text of the DECLARE constraint, an index can be used in the `set_constraints_to_violate_by_template_index` method

In [11]:
asp_gen: AspGenerator = AspGenerator(model, num_of_cases, num_min_events, num_max_events)

for id, constr_text in enumerate(model.serialized_constraints):
    print(f"{id} - {constr_text}")

asp_gen.set_constraints_to_violate_by_template_index(tot_negative_trace=10, violate_all=True, 
                                                 constraints_idx_list=[0, 3])
asp_gen.run()
asp_gen.to_xes(f'{model_name}.xes')

DEBUG:ASP generator:Distribution for traces uniform
DEBUG:ASP generator:traces: 10, events can have a trace min(3) max(10)
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8)]
DEBUG:Distributor:Custom_dist() min_mu:3 max_sigma:10 num_traces:10
DEBUG:Distributor:Probabilities sum 1
DEBUG:Distributor:Distribution result: [ 9 10  3  6  6 10  9  6  4  8]
INFO:ASP generator:Distribution result Counter({6: 3, 9: 2, 10: 2, 3: 1, 4: 1, 8: 1})
DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8

0 - Init[ER Registration] | |
1 - Chain Response[ER Registration, ER Triage] |
2 - Precedence[IV Liquid, Admission NC] | | |
3 - Response[CRP, Release B] |A.org:group is J |T.org:group is A |


DEBUG:ASP generator: Traces generated :[assigned_value(attr_name_0,attr_value_0,1), assigned_value(attr_name_0,attr_value_9,2), assigned_value(attr_name_0,attr_value_9,3), assigned_value(attr_name_0,attr_value_1,4), assigned_value(attr_name_0,attr_value_9,5), assigned_value(attr_name_0,attr_value_9,6), assigned_value(attr_name_1,attr_value_27,1), assigned_value(attr_name_1,attr_value_27,3), assigned_value(attr_name_1,attr_value_26,4), assigned_value(attr_name_1,attr_value_27,6), assigned_value(attr_name_2,20,1), assigned_value(attr_name_2,20,3), assigned_value(attr_name_2,20,4), assigned_value(attr_name_2,90,6), assigned_value(evt_val_4,50,1), assigned_value(evt_val_4,239,3), assigned_value(evt_val_4,2171,4), assigned_value(evt_val_4,3974,6), assigned_value(evt_val_5,101,2), assigned_value(evt_val_5,2,5), trace(evt_val_4,1), trace(evt_val_5,2), trace(evt_val_4,3), trace(evt_val_4,4), trace(evt_val_5,5), trace(evt_val_4,6)]
DEBUG:ASP generator: Clingo Result: SAT
DEBUG:ASP generator: Ge

exporting log, completed traces ::   0%|          | 0/10 [00:00<?, ?it/s]

## Setting up Rules for the Activation Conditions

Users can specify the number of activations of a DECLARE constraint in the synthetic cases. This can be done with the `set_activation_conditions` method by specifying an interval of activations for specific DECLARE constraints in the loaded model

In [12]:
asp_gen: AspGenerator = AspGenerator(model, num_of_cases, num_min_events, num_max_events)

asp_gen.set_activation_conditions({
'Response[CRP, Release B] |A.org:group is J |T.org:group is A |':
[2, 3]}) # activation should occur between 2 to 3 times

asp_gen.run()
asp_gen.to_xes(f'{model_name}.xes')

DEBUG:ASP generator:Distribution for traces uniform
DEBUG:ASP generator:traces: 10, events can have a trace min(3) max(10)
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8)]
DEBUG:Distributor:Custom_dist() min_mu:3 max_sigma:10 num_traces:10
DEBUG:Distributor:Probabilities sum 1
DEBUG:Distributor:Distribution result: [8 8 8 3 4 5 6 7 9 7]
INFO:ASP generator:Distribution result Counter({8: 3, 7: 2, 3: 1, 4: 1, 5: 1, 6: 1, 9: 1})
DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 

exporting log, completed traces ::   0%|          | 0/9 [00:00<?, ?it/s]

In addition, instead of giving the explicit text of the DECLARE constraints, an index can be used in the `set_activation_conditions_by_template_index` method

In [13]:
asp_gen: AspGenerator = AspGenerator(model, num_of_cases, num_min_events, num_max_events)

for id, constr_text in enumerate(model.serialized_constraints):
    print(f"{id} - {constr_text}")

asp_gen.set_activation_conditions_by_template_index({3: [2, 3]})
asp_gen.run()
asp_gen.to_xes(f'{model_name}.xes')

DEBUG:ASP generator:Distribution for traces uniform
DEBUG:ASP generator:traces: 10, events can have a trace min(3) max(10)
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1, 8)]
DEBUG:Distributor:Custom_dist() min_mu:3 max_sigma:10 num_traces:10
DEBUG:Distributor:Probabilities sum 1
DEBUG:Distributor:Distribution result: [ 3  6  8  7  5  9  3 10  4  7]
INFO:ASP generator:Distribution result Counter({3: 2, 7: 2, 6: 1, 8: 1, 5: 1, 9: 1, 10: 1, 4: 1})
DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 3 max_sigma: 10 num_traces: 10 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 8), Fraction(1, 8), Fraction(1, 8), Fraction(1,

0 - Init[ER Registration] | |
1 - Chain Response[ER Registration, ER Triage] |
2 - Precedence[IV Liquid, Admission NC] | | |
3 - Response[CRP, Release B] |A.org:group is J |T.org:group is A |


DEBUG:ASP generator: Traces generated :[assigned_value(attr_name_0,attr_value_1,1), assigned_value(attr_name_0,attr_value_9,2), assigned_value(attr_name_0,attr_value_9,3), assigned_value(attr_name_0,attr_value_1,4), assigned_value(attr_name_0,attr_value_9,5), assigned_value(attr_name_0,attr_value_9,6), assigned_value(attr_name_0,attr_value_9,7), assigned_value(attr_name_0,attr_value_9,8), assigned_value(attr_name_1,attr_value_28,1), assigned_value(attr_name_1,attr_value_27,2), assigned_value(attr_name_1,attr_value_29,3), assigned_value(attr_name_1,attr_value_29,4), assigned_value(attr_name_1,attr_value_26,6), assigned_value(attr_name_1,attr_value_28,7), assigned_value(attr_name_2,21,1), assigned_value(attr_name_2,23,2), assigned_value(attr_name_2,20,3), assigned_value(attr_name_2,23,4), assigned_value(attr_name_2,21,6), assigned_value(attr_name_2,24,7), assigned_value(attr_name_3,attr_value_148,1), assigned_value(attr_name_4,attr_value_147,1), assigned_value(attr_name_5,attr_value_148,

exporting log, completed traces ::   0%|          | 0/9 [00:00<?, ?it/s]